# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
with open("data/litw-win_new.txt", "r") as f:
    words = f.read().split()


def levenshtein_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_distance(s2, s1)

    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def correct_typo(word, words_list):
    min_distance = float("inf")
    closest_word = word

    for w in words_list:
        distance = levenshtein_distance(word, w)
        if distance < min_distance:
            min_distance = distance
            closest_word = w

    return closest_word


corrected_text = []
for word in text.split():
    if word in words:
        corrected_text.append(word)
    else:
        corrected_text.append(correct_typo(word, words))

corrected_text = " ".join(corrected_text)
print(text)
print(corrected_text)

с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий
с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [3]:
from nltk.stem.snowball import SnowballStemmer
from pymorphy2 import MorphAnalyzer

In [4]:
words = corrected_text.split()

stemmer = SnowballStemmer("russian")
morph = MorphAnalyzer()

stemmed_words = [stemmer.stem(word) for word in words]
lemmatized_words = [morph.parse(word)[0].normal_form for word in words]

print(stemmed_words)
print(lemmatized_words)

['с', 'величайш', 'усил', 'выбра', 'из', 'поток', 'убега', 'люд', 'кутуз', 'со', 'свит', 'уменьш', 'вдво', 'поеха', 'на', 'звук', 'выстрел', 'русск', 'оруд']
['с', 'великий', 'усилие', 'выбраться', 'из', 'поток', 'убегать', 'человек', 'кутузов', 'с', 'свита', 'уменьшиться', 'вдвое', 'поехать', 'на', 'звук', 'выстрел', 'русский', 'орудие']


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform([corrected_text])

print(vectorizer.get_feature_names_out())
print(X.toarray())

['вдвое' 'величайшим' 'выбравшись' 'выстрелов' 'звуки' 'из' 'кутузов'
 'людей' 'на' 'орудий' 'поехал' 'потока' 'русских' 'свитой' 'со'
 'убегающих' 'уменьшившейся' 'усилием']
[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import edit_distance
import random

In [3]:
# Загрузка предобработанных описаний рецептов из файла
df = pd.read_csv('data/recipes_sample.csv')
descriptions = df['description'].tolist()

In [4]:
# Получение набора уникальных слов
words = set()
for description in descriptions:
    if pd.isna(description):
        continue
    tokens = word_tokenize(description)
    words.update(tokens)

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [5]:
# Генерация 5 пар случайно выбранных слов и подсчет расстояния редактирования между ними
for i in range(5):
    word1, word2 = random.sample(words, 2)
    distance = edit_distance(word1, word2)
    print(f'Расстояние редактирования между словами "{word1}" и "{word2}" равно {distance}')

Расстояние редактирования между словами "bakeoff" и "sits" равно 7
Расстояние редактирования между словами "mits" и "taquito" равно 5
Расстояние редактирования между словами "spiciness" и "browns" равно 7
Расстояние редактирования между словами "-so" и "dessert.prep/cook" равно 15
Расстояние редактирования между словами "147254" и "gain" равно 6


C:\Users\alexa\AppData\Local\Temp\ipykernel_31408\1417854365.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  word1, word2 = random.sample(words, 2)


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [6]:
def find_closest_words(word, words, k):
    distances = [(edit_distance(word, w), w) for w in words]
    distances.sort(key=lambda x: x[0])
    closest_words = [w[1] for w in distances[:k]]
    return closest_words

In [7]:
word = "processing"
k = 5
closest_words = find_closest_words(word, words, 5)
print(f"{k} ближайших слов к '{word}': {closest_words}")

5 ближайших слов к 'processing': ['processing', 'pressing', 'proceeding', 'processer', 'processor/']


### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [8]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter

In [9]:
# Создание pd.DataFrame со столбцами word, stemmed_word и normalized_word
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

data = []
for word in words:
    stemmed_word = stemmer.stem(word)
    normalized_word = lemmatizer.lemmatize(word)
    data.append([word, stemmed_word, normalized_word])

df_words = pd.DataFrame(data, columns=['word', 'stemmed_word', 'normalized_word'])
df_words.set_index('word', inplace=True)

In [10]:
# Сравнение результатов стемминга и лемматизации
print(df_words.head(10))

                 stemmed_word normalized_word
word                                         
low-sugar           low-sugar       low-sugar
comstock             comstock        comstock
purdy                   purdi           purdy
lunchroom           lunchroom       lunchroom
cook.there          cook.ther      cook.there
1/2-centimetre  1/2-centimetr  1/2-centimetre
kall                     kall            kall
engine                  engin          engine
mizrahi               mizrahi         mizrahi
b4                         b4              b4


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [12]:
stop_words = set(stopwords.words('english'))

# Удаление стоп-слов из описаний рецептов
filtered_descriptions = []
for description in descriptions:
    if pd.isna(description):
        continue
    tokens = word_tokenize(description)
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_description = ' '.join(filtered_tokens)
    filtered_descriptions.append(filtered_description)

# Подсчет доли стоп-слов
total_words = sum(len(word_tokenize(description)) for description in descriptions if not pd.isna(description))
total_filtered_words = sum(len(word_tokenize(description)) for description in filtered_descriptions)
stop_words_ratio = (total_words - total_filtered_words) / total_words
print(f'Доля стоп-слов: {stop_words_ratio:.2%}')

Доля стоп-слов: 40.26%


In [13]:
# Сравнение топ-10 самых часто употребляемых слов до и после удаления стоп-слов
all_words = [word for description in descriptions if not pd.isna(description) for word in word_tokenize(description)]
all_filtered_words = [word for description in filtered_descriptions for word in word_tokenize(description)]

top_words = Counter(all_words).most_common(10)
top_filtered_words = Counter(all_filtered_words).most_common(10)

print('Топ-10 самых часто употребляемых слов до удаления стоп-слов:')
for word, count in top_words:
    print(f'{word}: {count}')

print('Топ-10 самых часто употребляемых слов после удаления стоп-слов:')
for word, count in top_filtered_words:
    print(f'{word}: {count}')

Топ-10 самых часто употребляемых слов до удаления стоп-слов:
.: 66166
the: 40257
,: 38544
a: 35030
and: 30425
i: 27799
this: 27132
to: 23508
it: 23212
is: 20501
Топ-10 самых часто употребляемых слов после удаления стоп-слов:
.: 66260
,: 38544
!: 16054
recipe: 15122
's: 7689
make: 6367
``: 5470
time: 5198
n't: 4798
use: 4645


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Выбор случайных 5 рецептов из набора данных
random_rows = df.sample(5)
random_descriptions = random_rows['description'].tolist()
random_names = random_rows['name'].tolist()

# Представление описаний рецептов в виде числовых векторов при помощи TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(random_descriptions)
tfidf_vectors = tfidf_matrix.toarray()

# Вывод результатов
for i, description in enumerate(random_descriptions):
    vector = tfidf_vectors[i]
    print(f'Описание рецепта: {description}')
    print(f'Вектор: {vector}')

Описание рецепта: fantastic chili recipe.   you can use dried ancho, pasilla, adobo or other peppers.   

pasilla is a good choice for a mild level of spice.  use bottled hot sauce per bowl to kick it up for the real heat lovers.


this recipe copied from the following url:  

http://www.greatchilirecipes.net/awful_good_texas_chili.html  

stored here as insurance in case the original site ever disappears.
Вектор: [0.         0.         0.         0.12326683 0.12326683 0.
 0.         0.12326683 0.12326683 0.         0.12326683 0.12326683
 0.         0.         0.         0.08255323 0.12326683 0.
 0.12326683 0.12326683 0.         0.         0.         0.12326683
 0.         0.         0.         0.         0.12326683 0.
 0.12326683 0.         0.12326683 0.12326683 0.         0.12326683
 0.1989018  0.0994509  0.12326683 0.         0.12326683 0.12326683
 0.12326683 0.12326683 0.12326683 0.12326683 0.08255323 0.
 0.12326683 0.0994509  0.08255323 0.12326683 0.12326683 0.
 0.12326683 0.     

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [24]:
# Вычисление близости между каждой парой рецептов
similarity_matrix = []
for i in range(len(tfidf_vectors)):
    row = []
    for j in range(len(tfidf_vectors)):
        similarity = 1 - cosine(tfidf_vectors[i], tfidf_vectors[j])
        row.append(similarity)
    similarity_matrix.append(row)

# Создание таблицы pd.DataFrame с результатами
df_similarity = pd.DataFrame(similarity_matrix, index=random_names, columns=random_names)

# Вывод результатов
print('Близость между каждой парой рецептов:')
print(df_similarity.to_string())

Близость между каждой парой рецептов:
                                        texas chili  salt and pepper shrimp  farfalle con pollo e spinaci  baked spanish risotto  cucumber in vinegar  pickled cucumbers
texas chili                                1.000000                0.000000                      0.075069               0.222029                                0.068761
salt and pepper shrimp                     0.000000                1.000000                      0.100230               0.087388                                0.105617
farfalle con pollo e spinaci               0.075069                0.100230                      1.000000               0.217201                                0.036577
baked spanish risotto                      0.222029                0.087388                      0.217201               1.000000                                0.031891
cucumber in vinegar  pickled cucumbers     0.068761                0.105617                      0.036577            

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

На основе представленной таблицы близости между каждой парой рецептов можно сделать вывод, что наиболее похожими являются рецепты “texas chili” и “baked spanish risotto”, так как значение близости между ними равно 0.222029, что является наибольшим значением в таблице. Это означает, что описания этих двух рецептов имеют наибольшее сходство среди всех выбранных рецептов.

Однако стоит отметить, что значения близости между всеми парами рецептов достаточно низкие, что указывает на то, что описания всех выбранных рецептов достаточно различны друг от друга.